In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_7987/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_7987/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_25_7,0.987967,0.830287,0.772505,0.908283,0.050087,1.134871,0.813738,0.198704,0.695646,0.223801,0.975158,0.227325,71.988000,143.742780,"Hidden Size=[8], regularizer=0.02, learning_ra..."
1,model_2_26_2,0.988405,0.827783,0.771355,0.979554,0.048266,1.151618,0.817851,0.143981,0.719311,0.219695,0.976061,0.223155,72.062055,143.816834,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_3_18_5,0.999248,0.825120,0.807986,0.997570,0.003132,1.169422,0.686824,0.009428,0.230353,0.055967,0.996789,0.056848,109.531992,216.076968,"Hidden Size=[12], regularizer=0.02, learning_r..."
3,model_2_26_6,0.989092,0.824981,0.770183,0.980531,0.045403,1.170351,0.822041,0.137100,0.731480,0.213081,0.977481,0.216437,72.184336,143.939116,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_3_17_8,0.999447,0.824762,0.808195,0.997571,0.002300,1.171817,0.686077,0.007101,0.165096,0.047959,0.997642,0.048715,110.149611,216.694587,"Hidden Size=[12], regularizer=0.02, learning_r..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,model_3_3_0,0.900132,0.703793,0.744234,0.846921,0.415704,1.980737,0.914860,0.561588,1.203398,0.644751,0.573895,0.654906,99.755561,206.300538,"Hidden Size=[12], regularizer=0.02, learning_r..."
334,model_1_5_0,0.913965,0.690419,0.692386,0.896163,0.358125,2.070173,1.100316,0.444947,1.248059,0.598435,0.882845,0.607860,36.053746,73.018330,"Hidden Size=[4], regularizer=0.02, learning_ra..."
335,model_1_4_4,0.910016,0.686413,0.697333,0.846853,0.374559,2.096957,1.082623,0.359759,1.246452,0.612013,0.877469,0.621651,35.964010,72.928593,"Hidden Size=[4], regularizer=0.02, learning_ra..."
336,model_1_4_3,0.909458,0.685751,0.697561,0.846528,0.376882,2.101385,1.081807,0.360522,1.245585,0.613907,0.876709,0.623576,35.951647,72.916230,"Hidden Size=[4], regularizer=0.02, learning_ra..."
